# Visualize the result of a list of GANs

In [ ]:
# general setup
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# for compatible with python 3
from __future__ import print_function
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Deep Convolutional GAN

In [ ]:
from dcgan import DCGAN
from utils import load_train_data, load_test_data

In [ ]:
# load cifar10 dataset
train_samples = load_train_data() / 255.0

In [ ]:
dcgan = DCGAN()
if False:
    dcgan.discriminator.summary()
    dcgan.generator.summary()
    dcgan.discriminator_generator.summary()

In [ ]:
dis_losses, gen_losses = dcgan.train(train_samples)

In [ ]:
from utils import viz_grid
# visualize
tracked_noise = np.random.normal(0, 1, [64, 64])
fig = plt.figure(figsize = (8, 8))   
ax1 = plt.subplot(111)
ax1.imshow(viz_grid(dcgan.generate(tracked_noise), 1))
plt.show()

plt.plot(dis_losses)
plt.title('discriminator loss')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()

plt.plot(gen_losses)
plt.title('generator loss')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()

## Conditional GAN (condition on the class labels)

In [ ]:
from cgan import CGAN
from utils import get_CIFAR10_data, to_categorical, get_mnist_32x32_1

In [ ]:
# load cifar10 data with labels
cifar10 = get_CIFAR10_data(subtract_mean=False, channels_first=False)
X_train = cifar10['X_train'] / 255.0
y_train = cifar10['y_train']
Y_train = to_categorical(y_train)

In [ ]:
# load mnist data
(x_train, y_train), (x_test, y_test) = get_mnist_32x32_1()
X_train = x_train.astype('float32') / 255.
X_test = x_test.astype('float32') / 255.
Y_train = to_categorical(y_train, 10)
Y_test = to_categorical(y_test, 10)

In [ ]:
print('x_train shape: {}, x_test shape: {}'.format(X_train.shape, X_test.shape))
print('y_train shape: {}, y_test shape: {}'.format(Y_train.shape, Y_test.shape))

In [ ]:
cgan = CGAN(num_channels=1)
if False:
    cgan.discriminator.summary()
    cgan.generator.summary()
    cgan.discriminator_generator.summary()

In [ ]:
dis_losses, gen_losses = cgan.train(X_train, Y_train, verbose=True)

In [ ]:
from utils import viz_grid
# visualize
tracked_noise = np.random.normal(0, 1, [64, 64])
labels = np.random.randint(low=0, high=10, size=64)
labels = to_categorical(labels, 10)
fig = plt.figure(figsize = (8, 8))
ax1 = plt.subplot(111)
ax1.imshow(viz_grid(cgan.generate(tracked_noise, labels), 1))
plt.show()

plt.plot(dis_losses)
plt.title('discriminator loss')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()

plt.plot(gen_losses)
plt.title('generator loss')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()

## Variational Autoencoder

In [ ]:
from vae import VariationalAutoencoder

In [ ]:
# load mnist dataset
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print('x_train shape: {}, x_test shape: {}'.format(x_train.shape, x_test.shape))

In [ ]:
vae = VariationalAutoencoder(input_dim=784)

In [ ]:
vae.model.summary()

In [ ]:
vae.train(x_train)

In [ ]:
# display a 2D plot of the digit classes in the latent space
x_test_encoded = vae.transformer(x_test)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()

In [ ]:
# Test the trained model: generation
# Sample noise vectors from N(0, 1)
w = h = 28
z = np.random.normal(size=[vae.batch_size, vae.n_z])
x_generated = vae.generator(z)

n = np.sqrt(vae.batch_size).astype(np.int32)
I_generated = np.empty((h*n, w*n))
for i in range(n):
    for j in range(n):
        I_generated[i*h:(i+1)*h, j*w:(j+1)*w] = x_generated[i*n+j, :].reshape(28, 28)

fig = plt.figure()
plt.imshow(I_generated, cmap='gray')

## Variational Recurrent Autoencoder

In [ ]:
from vrae import VariationalRecurrentAutoencoder

In [ ]:
# load mnist dataset
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
print('x_train shape: {}, x_test shape: {}'.format(x_train.shape, x_test.shape))

In [ ]:
vrae = VariationalRecurrentAutoencoder(input_dim=28, window_length=28)

In [ ]:
vrae.model.summary()

In [ ]:
vrae.train(x_train)

In [ ]:
# display a 2D plot of the digit classes in the latent space
x_test_encoded = vrae.transformer(x_test)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 2], c=y_test)
plt.colorbar()
plt.show()

In [ ]:
# Test the trained model: generation
# Sample noise vectors from N(0, 1)
w = h = 28
z = np.random.normal(size=[vrae.batch_size, vrae.n_z])
x_generated = vrae.generator(z)

n = np.sqrt(vrae.batch_size).astype(np.int32)
I_generated = np.empty((h*n, w*n))
for i in range(n):
    for j in range(n):
        I_generated[i*h:(i+1)*h, j*w:(j+1)*w] = x_generated[i*n+j, :].reshape(28, 28)

fig = plt.figure()
plt.imshow(I_generated, cmap='gray')

## Conditional Variational Recurrent Autoencoder

In [ ]:
from cvrae import ConditionalVariationalRecurrentAutoencoder

In [ ]:
# load mnist dataset
from keras.datasets import mnist
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
print('x_train shape: {}, x_test shape: {}'.format(x_train.shape, x_test.shape))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print('y_train shape: {}, y_test shape: {}'.format(y_train.shape, y_test.shape))

In [ ]:
model = ConditionalVariationalRecurrentAutoencoder(input_dim=28, window_length=28, num_classes=10)

In [ ]:
model.model.summary()

In [ ]:
model.load_weights()

In [ ]:
model.train(x_train, y_train, epochs=10)

In [ ]:
# Test the trained model: generation
# Sample noise vectors from N(0, 1)
digit = 5
w = h = 28
z = np.random.normal(size=[model.batch_size, model.n_z])
y = np.random.randint(low=digit, high=digit+1, size=model.batch_size)
y = to_categorical(y, 10)
x_generated = model.generator(z, y)

n = np.sqrt(model.batch_size).astype(np.int32)
I_generated = np.empty((h*n, w*n))
for i in range(n):
    for j in range(n):
        I_generated[i*h:(i+1)*h, j*w:(j+1)*w] = x_generated[i*n+j, :].reshape(28, 28)

fig = plt.figure()
plt.imshow(I_generated, cmap='gray')

## Recurrent Conditional GAN

In [ ]:
from rcgan_mnist import RCGANMNIST

In [ ]:
# load mnist data
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_test = (x_test.astype(np.float32) - 127.5) / 127.5
print('x_train shape: {}, x_test shape: {}'.format(x_train.shape, x_test.shape))
print('y_train shape: {}, y_test shape: {}'.format(y_train.shape, y_test.shape))

In [ ]:
model = RCGANMNIST(input_dim=28, window_length=28, num_classes=10)

In [ ]:
model.load_model()

In [ ]:
model.generator.summary()

In [ ]:
model.discriminator.summary()

In [ ]:
dis_losses, gen_losses = model.train(x_train, y_train, num_epoch=10)

In [ ]:
# Test the trained model: generation
# Sample noise vectors from N(0, 1)
w = h = 28
z = np.random.normal(-1, 1, size=[100, 64])
sampled_labels = np.array([[i] * 10 for i in range(10)]).reshape(-1, 1)
x_generated = model.generate(z, sampled_labels)
print(x_generated.shape)

In [ ]:
n = np.sqrt(100).astype(np.int32)
I_generated = np.empty((h*n, w*n))
for i in range(n):
    for j in range(n):
        I_generated[i*h:(i+1)*h, j*w:(j+1)*w] = x_generated[i*n+j, :].reshape(28, 28)

fig = plt.figure()
plt.imshow(I_generated, cmap='gray', vmin=-1, vmax=1, aspect='auto')

In [ ]:
model.save_model()